In [ ]:
from skimage import io
import numpy as np
from PIL import Image as im
import seaborn as sn
import matplotlib.pyplot as plt
import math
from IPython.display import clear_output
from skimage.morphology import binary_dilation, square
from skimage.filters import scharr_v, scharr_h
from skimage.color import rgb2gray
import scipy.stats as st
import copy
import time

In [ ]:
def display(img, title):
    io.imshow(img)
    plt.title('{}'.format(title))
    io.show()

In [ ]:
def removeObject(object_list, img_rgb, img_gray):
    
    filled_image = np.ones(img_gray.shape)
    for ob in object_list:
        filled_image[ob[0]:ob[1], ob[2]:ob[3]] = 0
    img_gray *= filled_image
    
    stat = io.imsave(outFolder+'mask.jpg', synth_img_gray*255.0)

    img_rgb[filled_image==0] = (0,0,0)
    display(img_gray, "Image for Synthesis")
    
    return filled_image, img_gray, img_rgb

In [ ]:
def getPriority(pixelList, img_gray, filled_image_pad, visited_mat, isophote_x, isophote_y, kernel):
    margin = math.floor(kernel/2)
    min_priority = 0.0
    p_pixel = (margin, margin)

    gradient_x = scharr_h(filled_image_pad[margin:img_gray.shape[0]+margin, margin:img_gray.shape[1]+margin])
    gradient_y = scharr_v(filled_image_pad[margin:img_gray.shape[0]+margin, margin:img_gray.shape[1]+margin])

    for pixel in pixelList:
        if pixel[0] < margin or pixel[0] >= img_gray.shape[0] + margin or pixel[1] < margin or pixel[1] >= img_gray.shape[1] + margin:
                continue
        
        # area = kernel*kernel
        # confidence = np.sum(visited_mat[pixel[0]-margin:pixel[0]+1+margin, pixel[1]-margin:pixel[1]+1+margin])/area
        confidence = np.sum(visited_mat[pixel[0]-margin:pixel[0]+1+margin, pixel[1]-margin:pixel[1]+1+margin])

        iso_grad_x = isophote_x[pixel[0]-margin, pixel[1]-margin]
        iso_grad_y = isophote_y[pixel[0]-margin, pixel[1]-margin]
        norm = math.sqrt(iso_grad_x*iso_grad_x + iso_grad_y*iso_grad_y)
        
        # avoid divide by zero error
        if norm != 0:
            iso_grad_x /= norm
            iso_grad_x /= norm
        
        p_vec_x = gradient_y[pixel[0] - margin, pixel[1]- margin]
        p_vec_y = gradient_x[pixel[0] - margin, pixel[1]- margin]
        norm = math.sqrt(p_vec_x*p_vec_x + p_vec_y*p_vec_y)
        
        # avoid divide by zero error
        if norm != 0:
            p_vec_x /= norm
            p_vec_y /= norm
        
        data = max(math.fabs(-p_vec_x*iso_grad_x + p_vec_y*iso_grad_y), math.fabs(p_vec_x*iso_grad_x + -p_vec_y*iso_grad_y))
        # data = math.fabs(p_vec_x*iso_grad_x + p_vec_y*iso_grad_y)
        priority = confidence * data

        if priority > min_priority:
            p_pixel = pixel
            min_priority = priority
        
    return p_pixel

In [ ]:
# Unfilled Neighbors
def getUnfilledNeighbors(filled, filled_image_pad, kernel_size):

    # returns index list, one for each axis (x[], y[])
    unfilled = np.nonzero(binary_dilation(filled_image_pad, square(3)) - filled_image_pad) 

    # combine axis to get index of unfilled pixels (x,y)
    coord = list(zip(unfilled[0], unfilled[1])) 
    
    return coord

In [ ]:
# Gaussian Kernel
def Gaussian2D(kernel, sigma):
    x = np.linspace(-sigma, sigma, kernel+1)
    kern1d = np.diff(st.norm.cdf(x))
    kern2d = np.outer(kern1d, kern1d)
    return kern2d/kern2d.sum()

In [ ]:
# Find Matches for each unfilles pixel
def FindMatches(pixel, template, patch_collection, patch_coords, filled_image_pad, kernel, sigma):
    '''
    validMask:      1s where Template is filled, 0s otherwise
    gaussMask:      2D Gaussian Kernel, same size as validMask
    templateMask:   gaussian weighted validMask
    totalWeight:    sum of values of templateMask
    reps:           total number of patches in patch_Collection
    dSSD:           normalized sum of squared differences
    '''
    half_kernel = math.floor(kernel/2)
    validMask = filled_image_pad[pixel[0]-half_kernel:pixel[0]+1+half_kernel, pixel[1]- half_kernel:pixel[1]+1+half_kernel]
    gaussMask = Gaussian2D(kernel, sigma)
    templateMask = validMask * gaussMask
    totalWeight = np.sum(templateMask)
    
    dSSD = templateMask * np.square(template - np.array(patch_collection))
    dSSD = np.sum(dSSD, axis=tuple(range(1,dSSD.ndim))) / totalWeight
    min_err = np.min(dSSD)

    bestMatch = np.argmin(dSSD)
    #bestMatches = [[i, d] for i, d in enumerate(dSSD) if d <= min_err*(1+errThreshold)]
    
    # return bestMatch, patch_coords[bestMatch]
    return patch_coords[bestMatch]

In [ ]:
# get example patches for image synthesis
def genPatches(img, filled_image_pad, visited_mat, kernel):
    
    margin = math.floor(kernel/2)

    # initialise example Patches array
    patch_collection = []
    patch_coords = []

    # collect sample patches, exclude patches that have any unfilled pixel
    for (x,y),v in np.ndenumerate(img):
        if v == 0:
            continue
        template = img[(x - margin):(x+1 + margin),(y-margin):(y+1+margin)]
        if template[template==0].shape[0] == 0:
            patch_collection.append(template)
            patch_coords.append((x - margin, y - margin))
        visited_mat[x, y] = 1
                
    return patch_collection, patch_coords, visited_mat

In [ ]:
#read sample image
def readImage(image, disp = False):
    img = io.imread('./'+image)
    img_gray = rgb2gray(img)
    
    if disp == True:
        display(img, "Read Image-{}".format(image.split('/')[1]))

    return img, img_gray

In [ ]:
imageFolder = 'Assignment-II-images-1'
outFolder = './synImages/'
images = ['test_im3.jpg'] #, 'test_im2.bmp']
# masks = ['masktest_im3.jpg']
kernel_size = [9]
errThreshold = 0.1
maxErrThreshold = 0.3
disp = False
f = open(outFolder+'log_criminsi.txt', "w")
f.close()

In [ ]:
blocks = ['person', 'sign', 'ground']
#person
person_coord = [(352,485,222,253)]

#sign
sign_coord = [(513,567,770,830),(566,664,788, 803)]

#ground
ground_coord = [(630, 664,3,390), (600, 630, 3, 435), (570,600,95,490), (540,570, 190,530),(510,540, 283, 570), (480, 510, 390, 610), (465,480, 465, 630),(440,465, 515,660)]

remove_blocks = [person_coord, sign_coord, ground_coord]

In [ ]:
for img_file in images:
    image = imageFolder+'/'+img_file
    img, img_gray = readImage(image, disp)

    # pixel values are in the range 0 to 1
    img_rgb = img/255.0
    img_gray = img_gray/255.0
    synth_img_gray = copy.copy(img_gray)
    synth_img_rgb = copy.copy(img_rgb)

    for i in range(len(remove_blocks)):
        filled_image, synth_img_gray, synth_img_rgb = removeObject(remove_blocks[i], synth_img_rgb, synth_img_gray)

        x,y = img_gray.shape
        isophote_x = scharr_h(img_gray)
        isophote_y = scharr_v(img_gray)
    
        output_dimx, output_dimy = np.shape(img_gray)
        print("Output Dimension: ({},{})".format(output_dimx, output_dimy))
        total_pixels = output_dimx * output_dimy
        
        for kernel in kernel_size:
            start = time.time()
            
            half_kernel = math.floor(kernel/2)
            margin = half_kernel
            sigma = kernel/6.4
    
            filled_image_pad = np.pad(filled_image, half_kernel, 'constant')
            synth_image_pad = np.pad(synth_img_gray, half_kernel, 'constant')
            img_rgb_pad = np.zeros((x+half_kernel * 2, y+half_kernel*2, 3))
            img_rgb_pad[half_kernel:x+half_kernel, half_kernel:y+half_kernel,] = synth_img_rgb
    
            visited_mat = copy.copy(filled_image_pad) #boundary_mat
    
            filled_pixels = np.sum(filled_image)
    
            patch_collection, patch_coords, visited_mat = genPatches(synth_image_pad, filled_image_pad, visited_mat, kernel)
    
            while filled_pixels < total_pixels:
                
                progress = 0
                pixelList = getUnfilledNeighbors(filled_image, filled_image_pad, kernel)
                # pixelList, order = getUnfilledNeighbors(filled_image, filled_image_pad, kernel)
                pixel = getPriority(pixelList, img_gray, filled_image_pad, visited_mat, isophote_x, isophote_y, kernel)
                print("pixel: {}".format(pixel))
                
                template = synth_image_pad[pixel[0]-half_kernel:pixel[0]+1+half_kernel, pixel[1]- half_kernel:pixel[1]+1+half_kernel]
                bestMatch = FindMatches(pixel, template, patch_collection, patch_coords, filled_image_pad, kernel, sigma)
                
                # synth_image[pixel[0], pixel[1]] = patch_collection[bestMatch, half_kernel, half_kernel]
                inver_visit = abs(filled_image_pad[pixel[0]-half_kernel:pixel[0]+1+half_kernel, pixel[1]- half_kernel:pixel[1]+1+half_kernel] - 1)
                inver_visit_rgb = np.ones((inver_visit.shape[0], inver_visit.shape[1], 3))
                for (x,y),v in np.ndenumerate(inver_visit):
                    if v == 1:
                        inver_visit_rgb[x,y] = np.array([1,1,1])
                    else:
                        inver_visit_rgb[x,y] = np.array([0,0,0])
    
                # synth_image_pad[pixel[0]:pixel[0]+2*kernel, pixel[1]+kernel:pixel[1]+2*kernel] += np.multiply(patch_collection[bestMatch], inver_visit)
                synth_image_pad[pixel[0]-half_kernel:pixel[0]+half_kernel+1, pixel[1]-half_kernel:pixel[1]+half_kernel+1] += np.multiply(synth_img_gray[bestMatch[0]-margin:bestMatch[0]+1+margin, bestMatch[1]-margin: bestMatch[1]+1+margin], inver_visit)
                img_rgb_pad[pixel[0]-half_kernel:pixel[0]+half_kernel+1, pixel[1]-half_kernel:pixel[1]+half_kernel+1] += np.multiply(synth_img_rgb[bestMatch[0]-margin:bestMatch[0]+1+margin, bestMatch[1]-margin: bestMatch[1]+1+margin], inver_visit_rgb)
                # filled_image[pixel[0], pixel[1]] = 1
                fp = np.sum(inver_visit)
                visited_mat[pixel[0]-half_kernel:pixel[0]+half_kernel+1, pixel[1]-half_kernel:pixel[1]+half_kernel+1] += inver_visit
                filled_image_pad[pixel[0]-half_kernel:pixel[0]+half_kernel+1, pixel[1]-half_kernel:pixel[1]+half_kernel+1] += inver_visit
                
                filled_pixels += fp
                
                clear_output()
                #if (total_pixels-filled_pixels)/total_pixels % 10 == 0:
                print(" Image = {}, kernel = {}, pixels_left = {}".format(img_file, kernel, total_pixels-filled_pixels))
                
                #if disp == True:
                # display_image_gray = synth_image_pad[half_kernel:half_kernel+img_gray.shape[0], half_kernel:half_kernel+img_gray.shape[1]]
                display_image_rgb = img_rgb_pad[margin:synth_image_pad.shape[0]-margin, margin:synth_image_pad.shape[1]-margin,]
                display_image_gray = synth_image_pad[margin:synth_image_pad.shape[0]-margin, margin:synth_image_pad.shape[1]-margin]
                display(display_image_gray, "Synthesizing Image: {}, kernel: {}".format(img_file.split('.')[0], kernel))
            
            synth_img_rgb = img_rgb_pad[margin:synth_image_pad.shape[0]-margin, margin:synth_image_pad.shape[1]-margin,]
            synth_img_gray = synth_image_pad[margin:synth_image_pad.shape[0]-margin, margin:synth_image_pad.shape[1]-margin]

            end = time.time()
    
        # save synthesized image
        final_image = io.imsave(outFolder+img_file.split('.')[0]+'_c_'+blocks[i]+'_k'+str(kernel)+'.'+img_file.split('.')[1], synth_img_gray*255.0)
        # final_image = im.fromarray(synth_img_rgb, 'RGB')
        # final_image.save(outFolder+img_file.split('.')[0]+blocks[i]+'_k'+str(kernel)+'.'+img_file.split('.')[1])

        # write log file
        writeLine = "File: {}, Block: {}, Kernel Size:{}, Time Taken:{} seconds\n".format(img_file, blocks[i], kernel, round(end-start,4))
        # append to the file
        with open(outFolder+"log_criminsi.txt", "a") as f:
            f.write(writeLine)
